<a href="https://colab.research.google.com/github/adanfernandez/Data-Mining/blob/main/Limpiar_textos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Primer ejercicio
## Preparación del entorno
Preparamos el entorno instalando las dependencias necesarias.


In [ ]:
!pip install spacy==3.0.5
!pip install ndjson

Instalamos el modelo que se usará en spaCy. Se instalará el modelo grande.

In [ ]:
!python -m spacy download es_core_news_lg

## Descarga de los datos de la máquina virtual
Importamos el archivo de datos desde el enlace Google Drive otorgado por el profesor de la asignatura. Para el primer ejercicio, que es el primero en el que nos encontramos, descargaremos el archivo con los datos procedentes de la API de twitter. El objetivo será limpiarlos.
La descarga de los tweets tardará más de dos minutos en producirse, realizándose en un directorio específico para el ejercicio 1.

In [ ]:
!mkdir ejercicio1
%cd ejercicio1

!gdown --id 1yBH-MYb3FapTWQ9ps3dkSjhaGOZKfNMK
!unzip tweets.zip

Eliminamos el fichero .zip para no agotar el espacio disponible.

In [ ]:
!rm tweets.zip

Se cargan los datos en memoria para posteriormente procesarlos. 

In [ ]:
import os
def cargar_tweets(directorio):
    lista = {}
    with os.scandir(directorio) as subdirectorios:
        for subdirectorio in subdirectorios:
            lista[directorio+subdirectorio.name] = os.listdir(directorio+subdirectorio.name)
    return lista

elementos_memoria = cargar_tweets("/content/ejercicio1/")
print(elementos_memoria)

## Limpieza de textos
Se usará la librería ndjson https://pypi.org/project/ndjson/ para leer los archivos, además de la biblioteca io.
Por otro lado, se usará el paquete spacy para obtener tanto los tokens como los sintagmas nominales de los tweets escritos en castellano.

In [ ]:
import json
import io
import ndjson
import spacy
from tqdm import tqdm

def limpiar_tweets(lista):
    json_data = []
    nlp = spacy.load("es_core_news_lg")
    for subdirectorio in lista:
        for fichero in lista[subdirectorio]:
          ruta = subdirectorio + "/" + fichero
          get_tweets(ruta, nlp)
        break

def get_tweets(ruta, nlp):
  english_tweets = []
  tweets = []
  with open(ruta) as f:
    datos = ndjson.load(f)
    for entrada in datos:
      if 'lang' in entrada and entrada['lang']=="es" or 'lang' not in entrada and entrada['user']['lang'] == 'es':
        hashtags = []
        for hashtag in entrada['entities']['hashtags']:
          hashtags.append(hashtag['text'])
        tweet_append = {
            "created_at": entrada['created_at'],
            "id_str": entrada['id_str'],
            "text": entrada['text'],
            "user": {
                "screen_name": entrada['user']['screen_name'],
                "description": entrada['user']['description']
            },
            "hashtags": hashtags,
            "urls": entrada['entities']['urls']
        }
        doc = nlp(entrada['text'])
        noum_chunks = []
        for chunk in doc.noun_chunks:
          noum_chunks.append(chunk.root.text)
        tweet_append['noum_chunks'] = noum_chunks
        tokens = [token.lemma_.lower() for token in doc if not token.is_punct and not token.is_stop]
        tweet_append['tokens'] = tokens
        tweets.append(tweet_append)
    return tweets

tweets_limpios = limpiar_tweets(elementos_memoria)